<center style="border-radius:10px;
padding: 3rem 2rem;
border: 3px solid #FCBE03;
">
<h1 style="color:#FCBE03;
font-size:3.0rem;
margin:0;
">Using 🤗 Transformers for the first time</h1>
<h2 style="color:#FCBE03;
font-size:2.0rem;
margin-top:1rem;
margin-bottom:2.5rem;
">Finetuning bert-base-uncased</h2>
<a href="https://kaggle.com/shreydan" style="color: white;
background-color: #FCBE03;
border-radius: 25px;
padding: 1rem 1.5rem;
text-decoration: none;
">@shreydan</a>
</center>

> **Hello everyone!** This is my first time using transformers and fine-tuning for a specific task. After many failed versions, referencing many notebooks and lot of time spent in docs later -- I was able to get a decent start with my transformers journey!

### I also made an EDA Notebook which you can check out here: [notebook](https://www.kaggle.com/code/shreydan/first-look-at-data-eda-read-some-essays)

<center style="border-radius:10px;
padding: 1rem;
border: 3px solid #731eb0;
">
<h3 style="color:#731eb0;
">📑️ first look at data | EDA | read some essays</h3>
</center>

## I finetuned more transformers after this, you can check out this public one:

### [DeBERTa-v3-base | 🤗️ Accelerate | Finetuning Notebook](https://www.kaggle.com/code/shreydan/deberta-v3-base-accelerate-finetuning)

- along with the Trainer class which you'll notice in this notebook, I integrated HuggingFace Accelerate -- which works as a helper to properly train transformers models. We do have to write our own custom training loops though!
- I also learnt how to use transformers offline, since for inference submissions we need to turn off the internet access.
- The reason I chose to use Accelerate was because my current training loops (the one in this notebook) are not very efficient, and even failed to execute with larger models such as deBERTa. Acclerate handles that with `prepare()` method to properly optimize the model, dataloaders, optimizer and scheduler.
- Another optimization technique I used was `gradient accumulation`, check out my other notebook for implementation using Accelerate and other transformer training optimization resources.

___

# Imports
<div style="width:100%;height:0;border-bottom: 3px solid #EAB209;margin-bottom: 1rem;"></div>

- I used AutoModel: which let's me import a pre-trained model for inference/fine-tuning.
- I used AutoTokenizer: a pre-trained tokenizer ofc but since its an AutoClass it  chooses the right tokenizer to use with the model. Pretty cool!!!
- I chose to write the training loop from scratch using Pytorch
- ~~Also if anyone can help me figure out the `tokenizers parallelism warning` and how to properly deal with it, please comment, I chose to set the environ flag for now.~~ I've noticed many notebooks just set this value to False.


In [ ]:
from transformers import AutoModel
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import gc

# ----------
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
import transformers
print('transformers version:',transformers.__version__)

# Config
<div style="width:100%;height:0;border-bottom: 3px solid #EDD608;margin-bottom: 1rem;"></div>

## We'll train two times, with and without freezing the encoder

- The model I used is the one in docs `bert-base-uncased`, uncased means the text has been converted to lower case during pre-tokenization
- max_length is for the maximum sequence length and the model's max_length = 512 and most of the essay lengths were in that region.
- The learning rate was crucial, I initally used lr=3e-4 but I wasn't getting any good results. After going through docs and searching for good learning rates for bert, order of 10**-5 seemed to be right and my results improved significantly
- batch_size set to 16 coz anything more I had CUDA out of memory issues
- scheduler: I used CosineAnnealingWarmRestarts because it was the one which was recommended in the docs
- dropout: I learnt that dropout is bad for regression. Previously I had used 0.3 and 0.5, let's just turn it off to 0.

In [ ]:
config = {
    'model': 'bert-base-uncased',
    'dropout': 0.,
    'max_length': 512,
    'batch_size': 16,
    'epochs': 4,
    'freeze_lr': 4e-4,
    'unfreeze_lr': 2e-5,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'scheduler': 'CosineAnnealingWarmRestarts'
}

# Tokenizer, Dataset and DataLoaders
<div style="width:100%;height:0;border-bottom: 3px solid #E4F008;margin-bottom: 1rem;"></div>

- After loading the tokenizer, I used the `encode_plus` method which converted the raw text to tokens as `input_ids` along with it we also get `token_type_ids` and `attention_mask`
- `token_type_ids` are used in cases such in sequence classification and question-answering. [docs](https://huggingface.co/transformers/v3.2.0/glossary.html#token-type-ids)
- `attention_mask` tells the model to what tokens to pay attention to and what to ignore.
- we also truncate till first 512 tokens and pad to max_length
- I used a simple pytorch dataset which takes the df, and generates samples as a dict.
- the inputs dict: `{'input_ids','token_type_ids','attention_mask'}`
- the targets dict: `{'labels'}` I treated this as a multiclass regression problem, the label is a vector of the 6 different types of scores in the dataset.
- then simple dataloaders for the dataset

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(config['model'])

In [ ]:
df = pd.read_csv('../input/feedback-prize-english-language-learning/train.csv')
test_df = pd.read_csv('../input/feedback-prize-english-language-learning/test.csv')

In [ ]:
test_df.head()

In [ ]:
print(df.columns.to_list())

In [ ]:
class EssayDataset:
    def __init__(self, df, config, tokenizer=None, is_test=False):
        self.df = df.reset_index(drop=True)
        self.classes = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
        self.max_len = config['max_length']
        self.tokenizer = tokenizer
        self.is_test = is_test

    def __getitem__(self,idx):
        sample = self.df['full_text'][idx]
        tokenized = tokenizer.encode_plus(sample,
                                          None,
                                          add_special_tokens=True,
                                          max_length=self.max_len,
                                          truncation=True,
                                          padding='max_length'
                                         )
        inputs = {
            "input_ids": torch.tensor(tokenized['input_ids'], dtype=torch.long),
            "token_type_ids": torch.tensor(tokenized['token_type_ids'], dtype=torch.long),
            "attention_mask": torch.tensor(tokenized['attention_mask'], dtype=torch.long)
        }

        if self.is_test == True:
            return inputs

        label = self.df.loc[idx,self.classes].to_list()
        targets = {
            "labels": torch.tensor(label, dtype=torch.float32),
        }

        return inputs, targets

    def __len__(self):
        return len(self.df)


In [ ]:
train_df, val_df = train_test_split(df,test_size=0.2,random_state=1357,shuffle=True)
print('dataframe shapes:',train_df.shape, val_df.shape)

In [ ]:
train_ds = EssayDataset(train_df, config, tokenizer=tokenizer)
val_ds = EssayDataset(val_df, config, tokenizer=tokenizer)
test_ds = EssayDataset(test_df, config, tokenizer=tokenizer, is_test=True)

In [ ]:
train_ds[0][0]['input_ids'].shape

In [ ]:
train_loader = torch.utils.data.DataLoader(train_ds,
                                           batch_size=config['batch_size'],
                                           shuffle=True,
                                           num_workers=2,
                                           pin_memory=True
                                          )
val_loader = torch.utils.data.DataLoader(val_ds,
                                         batch_size=config['batch_size'],
                                         shuffle=True,
                                         num_workers=2,
                                         pin_memory=True
                                        )

In [ ]:
print('loader shapes:',len(train_loader), len(val_loader))

# Model 1 : Unfreezed Encoder
<div style="width:100%;height:0;border-bottom: 3px solid #C3F307;margin-bottom: 1rem;"></div>

- the `encoder` returns 2 outputs, I turned off `return_dict` to unpack them. The first output is the last hidden state of the model, and the second output is pooled output of the model.
- the outputs we get (BaseModelOutputWithPoolingAndCrossAttentions):
    - last_hidden_state
    - pooler_output
    
    ```python
    >>> odict_keys(['last_hidden_state', 'pooler_output'])
    >>> torch.Size([16, 768]) # 768 is our hidden_size
    ```


- we can also choose the freeze the `encoder` to train the fc layers but eh this is fine.
- there are other pooling options we can use such as `mean pooling` which I'm yet to understand properly.
- the pooled output size can be obtained through `model.config.hidden_size`
- then simple fully-connected layers to get the logits we need


In [ ]:
class EssayModel(nn.Module):
    def __init__(self,config,num_classes=6):
        super(EssayModel,self).__init__()
        self.model_name = config['model']
        self.encoder = AutoModel.from_pretrained(self.model_name)
        self.dropout = nn.Dropout(config['dropout'])
        self.fc1 = nn.Linear(self.encoder.config.hidden_size,64)
        self.fc2 = nn.Linear(64,num_classes)

    def forward(self,inputs):
        _,outputs = self.encoder(**inputs, return_dict=False)
        outputs = self.dropout(outputs)
        outputs = self.fc1(outputs)
        outputs = self.fc2(outputs)
        return outputs

# Model 2 : Freezed Encoder
<div style="width:100%;height:0;border-bottom: 3px solid #C3F307;margin-bottom: 1rem;"></div>

- Freezing the model also optimizes the training, takes less GPU memory, and reduces overfitting, It's really easy to overfit transformers. I have 30+ versions to vouch for that, freezing the base model really helps

In [ ]:
class FrozenEssayModel(nn.Module):
    def __init__(self,config,num_classes=6):
        super(FrozenEssayModel,self).__init__()
        self.model_name = config['model']
        self.encoder = AutoModel.from_pretrained(self.model_name)

        # this is how you freeze a model: the base_model is generic term for the transformer name
        for param in self.encoder.base_model.parameters():
            param.requires_grad = False

        self.dropout = nn.Dropout(config['dropout'])
        self.fc1 = nn.Linear(self.encoder.config.hidden_size,64)
        self.fc2 = nn.Linear(64,num_classes)

    def forward(self,inputs):
        _,outputs = self.encoder(**inputs, return_dict=False)
        outputs = self.dropout(outputs)
        outputs = self.fc1(outputs)
        outputs = self.fc2(outputs)
        return outputs

# Trainer
<div style="width:100%;height:0;border-bottom: 3px solid #A2F606;margin-bottom: 1rem;"></div>

- **Loss Function:** [MCRMSE](https://www.kaggle.com/competitions/feedback-prize-english-language-learning/discussion/348985)
- **Optimizer:** AdamW, default weight-decay: 1e-2 -- huggingface docs recommend modifying the weight decay for params hence the `get_optim` method
- **Scheduler:** -- you can change the scheduler in the config and Trainer class will perform the scheduler.step() where it's supposed to.
    - ReduceLROnPlateau:: min_lr: 1e-7
    - **CosineAnnealingWithWarmRestarts::** T_0: 2, eta_min: 1e-7
    - StepLR:: step_size: 2
- **steps in every epoch:**
    - set model to train mode
    - train one epoch
        - get inputs, targets
        - move them to GPU
        - calculate loss
        - backprop
        - optim step
        - scheduler step
    - garbage collection and clear cuda cache
    - set model to eval mode
    - validate one epoch
        - get inputs, targets
        - move them to GPU
        - calculate loss
        - scheduler step if scheduler is ReduceLROnPlateau
    - garbage collection and clear cuda cache

In [ ]:
class Trainer:
    def __init__(self, model, loaders, config,lr='unfreeze'):
        self.model = model
        self.train_loader, self.val_loader = loaders
        self.config = config
        self.input_keys = ['input_ids','token_type_ids','attention_mask']

        self.loss_fn = nn.SmoothL1Loss()

        if lr == 'unfreeze':
            self.lr = self.config['unfreeze_lr']
        else:
            self.lr = self.config['freeze_lr']

        self.optim = self._get_optim()


        self.scheduler_options = {
            'CosineAnnealingWarmRestarts': torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(self.optim, T_0=5,eta_min=1e-7),
            'ReduceLROnPlateau': torch.optim.lr_scheduler.ReduceLROnPlateau(self.optim, 'min', min_lr=1e-7),
            'StepLR': torch.optim.lr_scheduler.StepLR(self.optim,step_size=2)
        }

        self.scheduler = self.scheduler_options[self.config['scheduler']]

        self.train_losses = []
        self.val_losses = []
        self.val_mcrmse = []

    def _get_optim(self):
        no_decay = ['bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in self.model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
            {'params': [p for n, p in self.model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
        optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=self.lr)
        return optimizer


    def mcrmse(self, outputs, targets):
        colwise_mse = torch.mean(torch.square(targets - outputs), dim=0)
        loss = torch.mean(torch.sqrt(colwise_mse), dim=0)
        return loss

    def train_one_epoch(self,epoch):

        running_loss = 0.
        progress = tqdm(self.train_loader, total=len(self.train_loader))

        for i,(inputs,targets) in enumerate(progress):

            self.optim.zero_grad()

            inputs = {k:inputs[k].to(device=config['device']) for k in inputs.keys()}
            targets = targets['labels'].to(device=config['device'])

            outputs = self.model(inputs)

            loss = self.loss_fn(outputs, targets)
            running_loss += loss.item()

            loss.backward()
            self.optim.step()

            if self.config['scheduler'] == 'CosineAnnealingWarmRestarts':
                self.scheduler.step(epoch-1+i/len(self.train_loader)) # as per pytorch docs

            del inputs, targets, outputs, loss

        if self.config['scheduler'] == 'StepLR':
            self.scheduler.step()

        train_loss = running_loss/len(self.train_loader)
        self.train_losses.append(train_loss)

    @torch.no_grad()
    def valid_one_epoch(self,epoch):

        running_loss = 0.
        running_mcrmse = 0.
        progress = tqdm(self.val_loader, total=len(self.val_loader))

        for (inputs, targets) in progress:

            inputs = {k:inputs[k].to(device=config['device']) for k in inputs.keys()}
            targets = targets['labels'].to(device=config['device'])

            outputs = self.model(inputs)

            loss = self.loss_fn(outputs, targets)
            running_loss += loss.item()

            running_mcrmse += self.mcrmse(outputs, targets).item()

            del inputs, targets, outputs, loss


        val_loss = running_loss/len(self.val_loader)
        self.val_losses.append(val_loss)

        self.val_mcrmse.append(running_mcrmse/len(self.val_loader))
        del running_mcrmse

        if config['scheduler'] == 'ReduceLROnPlateau':
            self.scheduler.step(val_loss)


    def test(self, test_loader):

        preds = []
        for (inputs) in test_loader:
            inputs = {k:inputs[k].to(device=config['device']) for k in inputs.keys()}

            outputs = self.model(inputs)
            preds.append(outputs.detach().cpu())

        preds = torch.concat(preds)
        return preds

    def fit(self):

        fit_progress = tqdm(
            range(1, self.config['epochs']+1),
            leave = True,
            desc="Training..."
        )

        for epoch in fit_progress:

            self.model.train()
            fit_progress.set_description(f"EPOCH {epoch} / {self.config['epochs']} | training...")
            self.train_one_epoch(epoch)
            self.clear()

            self.model.eval()
            fit_progress.set_description(f"EPOCH {epoch} / {self.config['epochs']} | validating...")
            self.valid_one_epoch(epoch)
            self.clear()

            print(f"{'-'*30} EPOCH {epoch} / {self.config['epochs']} {'-'*30}")
            print(f"train loss: {self.train_losses[-1]}")
            print(f"valid loss: {self.val_losses[-1]}\n\n")


    def clear(self):
        gc.collect()
        torch.cuda.empty_cache()

# Training
<div style="width:100%;height:0;border-bottom: 3px solid #7FF906;margin-bottom: 1rem;"></div>

## Unfreezed Encoder Model

In [ ]:
model = EssayModel(config).to(device=config['device'])
trainer_unfreeze = Trainer(model, (train_loader, val_loader), config, lr='unfreeze')

In [ ]:
trainer_unfreeze.fit()

___

In [ ]:
# freeing up GPU memory
del model
gc.collect()
torch.cuda.empty_cache()

___

## Freezed Encoder Model

In [ ]:
model = FrozenEssayModel(config).to(device=config['device'])
trainer_freeze = Trainer(model, (train_loader, val_loader), config, lr='freeze')

In [ ]:
trainer_freeze.fit()

# Results
<div style="width:100%;height:0;border-bottom: 3px solid #EAB209;margin-bottom: 1rem;"></div>

## Unfreezed Encoder Model

In [ ]:
losses_df = pd.DataFrame({'epoch':list(range(1,config['epochs'] + 1)),
                          'train_loss':trainer_unfreeze.train_losses,
                          'val_loss': trainer_unfreeze.val_losses,
                          'val_mcrmse': trainer_unfreeze.val_mcrmse
                         })
losses_df

## Freezed Encoder Model

In [ ]:
losses_df = pd.DataFrame({'epoch':list(range(1,config['epochs'] + 1)),
                          'train_loss':trainer_freeze.train_losses,
                          'val_loss': trainer_freeze.val_losses,
                          'val_mcrmse': trainer_freeze.val_mcrmse
                         })
losses_df

# Plots
<div style="width:100%;height:0;border-bottom: 3px solid #EDD608;margin-bottom: 1rem;"></div>

In [ ]:
plt.plot(trainer_unfreeze.train_losses, color='red')
plt.plot(trainer_unfreeze.val_losses, color='orange')
plt.title('MCRMSE Loss: Unfreezed Encoder')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [ ]:
plt.plot(trainer_freeze.train_losses, color='red')
plt.plot(trainer_freeze.val_losses, color='orange')
plt.title('MCRMSE Loss: Freezed Encoder')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

# Comparison
<div style="width:100%;height:0;border-bottom: 3px solid #0DFC25;margin-bottom: 1rem;"></div>

As you can see, I also tried the classic NLP methods, they were from scratch as well, even the embedding layer, could've used stuff like Word2Vec but I'm new to NLP so went with scratch implementations and ofcourse the results are not that good. Using a Byte-Pair encoding tokenizer improved the results slightly which was good to see, I used huggingface tokenizer for that as well but not pre-trained ofc.

## The notebooks:

- [LSTM + Embeddings](https://www.kaggle.com/code/shreydan/lstm-embeddings)
- [BPE Tokenizer + LSTM + Embeddings](https://www.kaggle.com/code/shreydan/bpe-tokenizer-lstm-embeddings)

![lstm](https://i.imgur.com/6vAeO6a.jpeg)


# Closing
<div style="width:100%;height:0;border-bottom: 3px solid #EAB209;margin-bottom: 1rem;"></div>

## Thank you for taking the time to go through my notebook!

- Would appreciate feedback in the comments, please do let me know what I could've done better, how I can improve my training loops and especially using regarding usage of transformers.
- **Upvote if you liked this notebook, really helps a lot as I'm currently new to NLP.**
- **Follow if you like my work! :)**

## References:

- [FB3: RoBERTa PyTorch Baseline - KFolds + W&B 🚀 by @heyytanay](https://www.kaggle.com/code/heyytanay/fb3-roberta-pytorch-baseline-kfolds-w-b) - his work is amazing, I learnt a lot from just reading his notebooks since I joined kaggle! Do checkout his work.
- [HuggingFace AutoClass](https://huggingface.co/docs/transformers/main/autoclass_tutorial)
- [HuggingFace training & Finetuning with Pytorch](https://huggingface.co/transformers/v3.3.1/training.html#pytorch)
- [tez: fb3 training kernel by @abhishek](https://www.kaggle.com/code/abhishek/tez-fb3-training-kernel) - he basically taught me Pytorch, creating datasets andwriting training loops. Checkout his youtube tutorials, pretty cool!
- [HuggingFace training](https://huggingface.co/docs/transformers/main/training)

___

Matthew 5:9